In [ ]:
import pandas as pd
import requests
import xmltodict
import json
import matplotlib.pyplot as plt

In [ ]:
state_list = ["AL", "AK", "AZ", "AR", "CA","CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY","NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [ ]:
def get_state_results(state, skip):
    url = 'https://npiregistry.cms.hhs.gov/api/?version=2.1'
    parameters = {'version':'2.1', 'enumeration_type':'NPI-2', 'state':state,'taxonomy_description':'General Acute Care Hospital', 'address_purpose':'LOCATION','skip':skip,'limit':200}
    response =  requests.get(url, params=parameters)
    results = response.json()
    return results
def get_all_state_results(state):
    state_results = []
    skip_count = 0
    results = get_state_results(state, skip_count)
    state_results+=results['results']
    last_result = 'None'
    first_result = results['results'][0]['basic']['organization_name']
    while results['result_count']==200 and first_result != last_result:
        print(skip_count)
        skip_count += 200
        results = get_state_results(state, skip_count)
        last_result=first_result
        first_result = results['results'][0]['basic']['organization_name']
        print(first_result)
        state_results+=results['results']
    return(state_results)

In [ ]:
def get_hospitals(state_results): 
    npi_dicts = []
    addresses = []
    names = []
    for result in state_results:
        name = result['basic']['organization_name']
        if name not in names:
            names.append(name)
        else:
            print (f'{name} is repeat name')       
        if result['basic']['organizational_subpart']=='NO':
            for tax in result['taxonomies']:
                if tax['desc'] == 'General Acute Care Hospital' and tax['primary']==True:
                    for address in result['addresses']:
                        if address['address_purpose']=='LOCATION':
                            add = address['address_1']
                            if add not in addresses:
                                addresses.append(add) 
                                npi_dict = {
                                    'npi': result['number'],
                                    'name': name,
                                    'address_1': add,
                                    'address_2': address['address_2'],
                                    'city': address['city'],
                                    'state': address['state'],
                                    'postal_code': address['postal_code'],
                                    'date': result['basic']['last_updated']
                                    }
                                npi_dicts.append(npi_dict)
                            else:
                                print (f'{name} :  {add} is repeat address')
    return npi_dicts

In [ ]:
npi_dict_list = []
for state in state_list:
    print(state)
    state_results = get_all_state_results(state)
    npi_dict_list += get_hospitals(state_results)
    print(f'{len(npi_dict_list)} hospitals!')
    print('---')

In [ ]:
xx= pd.DataFrame(npi_dict_list).sort_values('name')

In [ ]:
len(state_results)

In [ ]:
state_results

In [ ]:
state_results = get_all_state_results('NY')

In [ ]:
get_state_results('TX', 900)

In [ ]:
get_state_results('AZ',0)

In [ ]:
xx.sort_values('name')

In [ ]:
xx[3305:]

In [52]:
import pandas as pd
ppd = pd.read_csv('../../Data/Scorecard/ppd_data_20200704.csv')

In [ ]:
ppd[(ppd['LAST_NAME']=='WARREN')&(ppd['FIRST_NAME']=="DE'ANDRE")][['ME', 'MAILING_NAME',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 
       'TOP_CD', 'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'MEDSCHOOL_STATE', 'MEDSCHOOL_ID',
       'MEDSCHOOL_GRAD_YEAR', 'MD_DO_CODE']]

In [ ]:
for thing in ppd.columns:
    print (thing)

In [ ]:
ppd['FIPS_COUNTY']

In [3]:
geo = pd.read_excel('../../Data/geocodes.xlsx')

In [ ]:
geo.columns

In [ ]:
pd.merge(geo, ppd, left_on='County Code (FIPS)', right_on='FIPS_COUNTY')

In [12]:
geo.groupby(['County Code (FIPS)']).count()[50:100]

,Summary Level,State Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
County Code (FIPS),,,,,,
75,256,256,256,256,256,256
77,251,251,251,251,251,251
78,2,2,2,2,2,2
79,270,270,270,270,270,270
81,273,273,273,273,273,273
83,253,253,253,253,253,253
85,220,220,220,220,220,220
86,1,1,1,1,1,1
87,213,213,213,213,213,213


In [6]:
geo.columns

Index(['Summary Level', 'State Code (FIPS)', 'County Code (FIPS)',
       'County Subdivision Code (FIPS)', 'Place Code (FIPS)',
       'Consolidtated City Code (FIPS)',
       'Area Name (including legal/statistical area description)'],
      dtype='object')

In [13]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [ ]:
pd.merge(geo, ppd, left_on=['State Code (FIPS)','County Code (FIPS)'], right_on=['FIPS_STATE','FIPS_COUNTY'])

In [53]:
ppd['FIPS_COUNTY'] = ppd.FIPS_COUNTY.fillna('00')
ppd['FIPS_STATE'] = ppd.FIPS_STATE.fillna('00')

In [54]:
ppd_codes = []
for row in ppd.itertuples():
    state = str(int(row.FIPS_STATE))
    county = str(int(row.FIPS_COUNTY))
    if len(state)<2:
        state = '0'+state
    if len(county)==1:
        county = '00'+county
    elif len(county)==2:
        county = '0'+county
    ppd_codes.append(state+county)

In [16]:
geo = geo.rename(columns={'State Code (FIPS)':'FIPS_STATE','County Code (FIPS)':'FIPS_COUNTY'})

In [19]:
geo_codes = []
for row in geo.itertuples():
    state = str(int(row.FIPS_STATE))
    county = str(int(row.FIPS_COUNTY))
    if len(state)<2:
        state = '0'+state
    if len(county)==1:
        county = '00'+county
    elif len(county)==2:
        county = '0'+county
    geo_codes.append(state+county)

In [55]:
geo['CODE'] = geo_codes
ppd['CODE'] = ppd_codes

In [56]:
MERGE = pd.merge(ppd, GEO, on='CODE')

In [39]:
GEO = geo[geo['Summary Level'].isin([50, 40])]

In [57]:
MERGE.drop_duplicates('ME')

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,CODE,Summary Level,FIPS_STATE_y,FIPS_COUNTY_y,County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,102000013,A,1,2.0,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,NaN,...,NaN,NaN,01117,50,1,117,0,0,0,Shelby County
1,102000633,A,1,1.0,JASON M JACK MD,JACK,JASON,MICHAEL,NaN,NaN,...,NaN,NaN,01117,50,1,117,0,0,0,Shelby County
2,102000927,A,1,1.0,CLIFTON R VANCE MD,VANCE,CLIFTON,RODGERS,NaN,HEART SOUTH CARDIOVASCULAR GROUP P C,...,NaN,NaN,01117,50,1,117,0,0,0,Shelby County
3,102002237,A,1,2.0,JAMES R TRIMM MD,TRIMM,JAMES,RANDAL,NaN,NaN,...,NaN,NaN,01117,50,1,117,0,0,0,Shelby County
4,102010418,A,1,2.0,CHRIS A PHILLIPS MD,PHILLIPS,CHRIS,ARIC,NaN,NaN,...,NaN,NaN,01117,50,1,117,0,0,0,Shelby County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272747,74801655053,A,1,NaN,MARIA M SERAPIO MD,SERAPIO,MARIA,M,NaN,MEDICAL DENTAL FACILITY,...,NaN,NaN,28041,50,28,41,0,0,0,Greene County
1272748,74810880035,A,1,1.0,SHAWN E BAILEY MD,BAILEY,SHAWN,EDWIN,NaN,NaN,...,NaN,NaN,17155,50,17,155,0,0,0,Putnam County
1272749,74811720325,A,1,1.0,ESTELA L CHAN MD,CHAN,ESTELA,LAO,NaN,NaN,...,NaN,NaN,31181,50,31,181,0,0,0,Webster County
1272750,78103820013,A,1,2.0,ADELE LUKASZEWICZ MD,LUKASZEWICZ,ADELE,NaN,NaN,NaN,...,NaN,NaN,30079,50,30,79,0,0,0,Prairie County


In [58]:
len(MERGE)

1272752

In [43]:
ppd.groupby('CODE').count().sort_values('ME', ascending=False)

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
CODE,,,,,,,,,,,,,,,,,,,,,
31031,55640,55640,55640,55147,55640,55639,55640,43757,1330,10134,...,42289,42290,41797,41938,12216,9990,12215,1455,1139,1455
37037,49338,49338,49338,48927,49338,49336,49337,37572,1072,7793,...,38564,38564,38223,38350,10618,8388,10618,1304,981,1304
03003,44485,44485,44485,44093,44485,44484,44484,34983,1171,6606,...,34312,34312,33947,34063,10287,8535,10287,1216,947,1216
01001,39685,39685,39685,39185,39685,39685,39685,30841,985,8336,...,30696,30697,30413,30525,8204,6953,8204,916,757,916
61061,39372,39372,39372,38775,39372,39371,39372,29826,742,9504,...,30015,30014,29626,29796,8077,6400,8078,909,688,910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359359,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
327327,1,1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
393393,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
ppd[ppd.FIPS_COUNTY=='00']

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,CODE
2432,102150228,A,1,2.0,BRANDON S WITHERS MD,WITHERS,BRANDON,SCOTT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000
3319,102200594,A,1,NaN,CLARENCE H MULLINS IV MD,MULLINS,CLARENCE,HADDON,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000
4118,102700872,A,1,1.0,WILLIAM L ALBRITTON MD,ALBRITTON,WILLIAM,L,NaN,#B103 HLTH SCI BLDG,...,S7N 5,NaN,NaN,ALBRITTON,LEONARD,WM,NaN,NaN,NaN,00000
5252,102801911,A,1,2.0,WADE A HARRIS MD,HARRIS,WADE,ALLEN,NaN,NaN,...,29690,9680.0,R004,NaN,NaN,NaN,NaN,NaN,NaN,00000
5780,102840763,A,1,2.0,MICHAEL G KIDD MD,KIDD,MICHAEL,GENE,NaN,667 CMR 402,...,28360,8288.0,R008,NaN,NaN,NaN,NaN,NaN,NaN,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282468,95701870078,A,1,2.0,GORAN TORLAKOVIC MD,TORLAKOVIC,GORAN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000
1282495,95701880081,A,1,2.0,RATKO J SARAZIN MD,SARAZIN,RATKO,J,NaN,DIEPPE PROV NEW BRUNSWICK,...,32501,1857.0,C023,NaN,NaN,NaN,NaN,NaN,NaN,00000
1282812,95702620091,A,1,2.0,VELJKO K ZIVKOVICH MD,ZIVKOVICH,VELJKO,KRSTA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000
1283502,95708780091,A,1,NaN,LJILJANA A MIJATOVIC MD,MIJATOVIC,LJILJANA,A,NaN,NaN,...,14215,1129.0,C024,MIJATOVIC,ALEKSANDAR,L,NaN,NaN,NaN,00000


In [48]:
len(ppd)

1283795

In [51]:
ppd[['FIPS_STATE','FIPS_COUNTY', 'CODE']]

,FIPS_STATE,FIPS_COUNTY,CODE
0,117,117,117117
1,183,183,183183
2,119,119,119119
3,13,13,13013
4,71,71,71071
...,...,...,...
1283790,135,135,135135
1283791,3,3,03003
1283792,81,81,81081
1283793,81,81,81081


In [66]:
pd.merge(GEO, ppd, on=['FIPS_STATE','FIPS_COUNTY'])

,Summary Level,FIPS_STATE,FIPS_COUNTY,County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description),CODE_x,ME,RECORD_ID,...,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,CODE_y
0,50,1,1,0,0,0,Autauga County,01001,102001141,A,...,36701,2202.0,R009,NaN,NaN,NaN,NaN,NaN,NaN,01001
1,50,1,1,0,0,0,Autauga County,01001,102040813,A,...,36066,7263.0,R016,NaN,NaN,NaN,NaN,NaN,NaN,01001
2,50,1,1,0,0,0,Autauga County,01001,102061543,A,...,36003,2535.0,R401,NaN,NaN,NaN,NaN,NaN,NaN,01001
3,50,1,1,0,0,0,Autauga County,01001,102100760,A,...,35010,3369.0,C005,NaN,NaN,NaN,NaN,NaN,NaN,01001
4,50,1,1,0,0,0,Autauga County,01001,102101081,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272747,50,72,153,0,0,0,Yauco Municipio,72153,84710640326,A,...,00698,3541.0,C002,MARIN,CECILIO,RADAMES,NaN,NaN,NaN,72153
1272748,50,72,153,0,0,0,Yauco Municipio,72153,84710710201,A,...,00698,3530.0,C002,AROCHO,ERNESTO,LUIS,NaN,NaN,NaN,72153
1272749,50,72,153,0,0,0,Yauco Municipio,72153,84710920027,A,...,00717,1321.0,C050,BENITEZ,NaN,VICTOR,NaN,NaN,NaN,72153
1272750,50,72,153,0,0,0,Yauco Municipio,72153,84714840010,A,...,00653,2810.0,C001,DEL RODRIGUEZ,NaN,MILAGROS,NaN,NaN,NaN,72153


In [61]:
geo.dtypes

Summary Level                                                int64
FIPS_STATE                                                   int64
FIPS_COUNTY                                                  int64
County Subdivision Code (FIPS)                               int64
Place Code (FIPS)                                            int64
Consolidtated City Code (FIPS)                               int64
Area Name (including legal/statistical area description)    object
CODE                                                        object
dtype: object

In [64]:
ppd['FIPS_STATE'] = ppd.FIPS_STATE.astype(int)
ppd['FIPS_COUNTY'] = ppd.FIPS_COUNTY.astype(int)